In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchaudio
import sys
import torch
import numpy as np
import pandas as pd
import os
import pickle
import re
import torchaudio.transforms as T
import math
import librosa
import librosa.display
import matplotlib.patches as patches
from glob import glob
import util
from util import audio_file
from util import *
from Losses import FocalLoss,WeightedFocalLoss
from StackedLSTM2 import StackedLSTM
from AttentionLSTM import Attention_LSTM
from Toy_Model import ToyModel
import time
torch.manual_seed(1)

my_dataset = "ATC0"
my_model = "Attention_LSTM"
verbose = 1

'''if my_dataset == "ATC0":
    input_list, labels_list = process_atc0_files(2)
    
elif my_dataset == "BPC":
    input_list, labels_list = load_data_limit(5)

n_samples = input_list.size()[0]
train_split = 4*n_samples//5
test_samples = n_samples - train_split
    
test_input_list = input_list[train_split:]
test_labels_list = labels_list[train_split:]
input_list = input_list[:train_split]
labels_list = labels_list[:train_split]

if my_model == "Attention_LSTM":
    model = Attention_LSTM()
    save_filepath = '/project/graziul/ra/ajays/LSTM_model_predictions.txt'
elif my_model == "Vanilla_LSTM":
    model = ToyModel()
    save_filepath = '/project/graziul/ra/ajays/toy_model_predictions.txt'
'''
sample_size = 30
batch_size = model.batch_size
num_samples = input_list.size()[0]//batch_size
training_steps = 5
idx = 0
flag = 0
num_segments = 30
val_size = 30
gammas = [0.0001,0.001,0.005,0.1,1,10]

for gamma in gammas:
    train_loss_list = []
    loss_fn = FocalLoss(gamma = gamma)
    if my_model == "Attention_LSTM":
        model = Attention_LSTM()
        save_filepath = '/project/graziul/ra/ajays/LSTM_model_predictions.txt'
    elif my_model == "Vanilla_LSTM":
        model = ToyModel()
        save_filepath = '/project/graziul/ra/ajays/toy_model_predictions.txt'
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    fer_list = []

    test_loss_list = []

    for step in range(training_steps):
        start_time = time.time()
        input_batch = input_list[idx*batch_size:(idx+1)*batch_size]
        labels_batch = labels_list[idx*batch_size:(idx+1)*batch_size]
        idx = (idx+1)%num_samples
        print(step)
        optimizer.zero_grad()
        output_hat = model(input_batch)
        #print(output_hat)
        print(labels_batch)
        print(output_hat)
        loss = loss_fn(output_hat, labels_batch)
        loss.backward()
        #for param in model.parameters():
        #    print(param.grad)
        print(loss)
        train_loss_list.append(loss.item())
        optimizer.step()
        end_time = time.time()
        step_time = end_time - start_time
        print("Time Taken for Step = " + str(step_time))

    plt.plot(list(range(training_steps)),train_loss_list, label = str(gamma))
plt.legend()
plt.show()

0
tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])
tensor([[0.5324, 0.5431, 0.5024,  ..., 0.5312, 0.5380, 0.5326],
        [0.5259, 0.5436, 0.5022,  ..., 0.5245, 0.5241, 0.5058]],
       grad_fn=<SqueezeBackward0>)
tensor(0.7259, grad_fn=<MeanBackward0>)
Time Taken for Step = 2.2342209815979004
1
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0.4094, 0.2236, 0.4213,  ..., 0.4120, 0.3607, 0.2848],
        [0.3454, 0.1940, 0.3944,  ..., 0.4257, 0.3473, 0.2807]],
       grad_fn=<SqueezeBackward0>)
tensor(0.2196, grad_fn=<MeanBackward0>)
Time Taken for Step = 2.3978078365325928
2
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[0.2954, 0.2022, 0.3237,  ..., 0.2971, 0.1834, 0.2548],
        [0.3020, 0.2067, 0.3266,  ..., 0.2910, 0.1841, 0.2497]],
       grad_fn=<SqueezeBackward0>)
tensor(0.0327, grad_fn=<MeanBackward0>)
Time Taken for Step = 2.4273271560668945
3
tensor([[0., 0

In [1]:
#Threshold on ATC0
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchaudio
import sys
import torch
import numpy as np
import pandas as pd
import os
import pickle
import re
import torchaudio.transforms as T
import math
import librosa
import librosa.display
import matplotlib.patches as patches
from glob import glob
import util
from util import audio_file
from util import *
from Losses import FocalLoss,WeightedFocalLoss
from StackedLSTM2 import StackedLSTM
from AttentionLSTM import Attention_LSTM
from Toy_Model import ToyModel
import time

zcr_list = []
rms_list = []
autocorrelation_list = []
labels_list = []
audio_file_arr = []
frame_size = 25
frame_hop = 10
idx = 0
transcript_list_arr = []
min_zcr_t = 0
min_rms_t = 0
max_zcr_t = 10
max_rms_t = 10
zcr_t_arr = np.linspace(min_zcr_t,max_zcr_t,20)
#zcr_threshold = 1
rms_t_arr = np.linspace(min_rms_t,max_rms_t,20)

input_list = []
labels_list = []
txtfile = open('/project/graziul/ra/ajays/stats_atc0.txt','w')
txtfile.truncate(0)
#paths = ['/project/graziul/data/corpora/atc0_comp/atc0_bos/data/audio/', '/project/graziul/data/corpora/atc0_comp/atc0_dca/data/audio/', '/project/graziul/data/corpora/atc0_comp/atc0_dfw/data/audio/']
paths = ['/project/graziul/data/corpora/atc0_comp/atc0_bos/data/audio/']
for idx,path in enumerate(paths):
    for fpath in glob(path + '*.sph'):
        if(idx > k):
            break
        filename = fpath[-12:-4]
        label_file = path[:-6] + 'transcripts/' + filename + '.txt'
        waveform, sample_rate = torchaudio.load(file_name)
        effects = [['rate', '22050']]
        x,sr = torchaudio.sox_effects.apply_effects_tensor(waveform, sample_rate, effects)
        with open(label_file, 'r') as f:
            label = f.read()
        y = process_atc_label_line(label)
        
        input_list.append(x)
        labels_list.append(y)
        idx = idx+1
        zcr = np.array(np.squeeze(get_zcr(x,sr, frame_length=sr*frame_size,hop_length=sr*frame_hop)))
        zcr_list.append(zcr)
        rms = np.array(np.squeeze(librosa.feature.rms(x,frame_length=sr*frame_size,hop_length=sr*frame_hop)))
        rms_list.append(rms)
        get_plots(x,zcr,rms)
        frame_error_rate_list = []
        for zcr_threshold in zcr_t_arr:
            for rms_threshold in rms_t_arr:
                frame_error_rate = process_single_file(x,label,sr,zcr_threshold,rms_threshold)
                frame_error_rate_list.append(frame_error_rate)
                writeline = audio_file + ' ' + str(zcr_threshold) + ' ' + str(rms_threshold) + ' ' + str(frame_error_rate) + '\n'
                txtfile.write(writeline)
        frame_error_rate_list = np.array(frame_error_rate_list)
        get3dplots(frame_error_rate_list, zcr_t_arr, rms_t_arr,idx)       
        
    if(idx>k):
        break

NameError: name 'k' is not defined